#### Dependencies

In [2]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager

#### Setup splinter

In [3]:
# Setup Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)  



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\vmuty\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


#### NASA Mars News

* Scraped the [Mars News Site](https://redplanetscience.com/) and collected the latest News Title and Paragraph Text. Stored the text to variables. Also scraped the date of the article which is ofcourse, the current date.

* This URL loaded noticibly slow, may be because of it's dynamic content.
Using 'time' to make the execution wait for 2 seconds.

* In case that fails, it enters a while loop to make sure it does not go further unless this page has loaded completely and has been scraped.

In [4]:
scrape_data = {}

# Scrape NASA Mars News
url_redplanetscience = 'https://redplanetscience.com/'

browser.visit(url_redplanetscience)
time.sleep(2)    

In [6]:
# If the website did not entirely load and failed to fetch info, only then it does the following
while len(browser.find_by_xpath('//div[@class="list_text"]')) == 0  :
    browser.visit(url_redplanetscience)
    time.sleep(2) 
    browser.reload()

html_redplanetscience = browser.html
soup_redplanetscience = BeautifulSoup(html_redplanetscience, 'html.parser')

results_redplanetscience = soup_redplanetscience.find('div', class_='list_text')
news_date  = results_redplanetscience.find('div', class_='list_date').text
news_title = results_redplanetscience.find('div', class_='content_title').text
news_para  = results_redplanetscience.find('div', class_='article_teaser_body').text

scrape_data["news_date"]  = news_date
scrape_data["news_title"] = news_title
scrape_data["news_para"]  = news_para


print (f"{news_date} :\n{news_title}\n{news_para}")


August 16, 2021 :
NASA Engineers Checking InSight's Weather Sensors
An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.


### JPL Mars Space Images - Featured Image
* Visited url for the Featured Space Image site [here](https://spaceimages-mars.com).
* Used splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.
* Searched for full size `.jpg` image.
* Saved the url string for this image.

In [10]:
# Code for featured_image_url 
url_spaceimages = 'https://spaceimages-mars.com/'
browser.visit(url_spaceimages)

html_spaceimages = browser.html
soup_spaceimages = BeautifulSoup(html_spaceimages, 'html.parser')
results_spaceimages = soup_spaceimages.find('div', class_='floating_text_area')
featured_image = results_spaceimages.find('a').text.strip().upper()

if (featured_image != "" and featured_image == "FULL IMAGE"):
    featured_image = results_spaceimages.find('a').get('href')
else:
    featured_image == soup_spaceimages.find('a', class_='showimg fancybox-thumbs').get('href')

featured_image_url = url_spaceimages + featured_image
scrape_data["featured_image_url"]  = featured_image_url
featured_image_url


'https://spaceimages-mars.com/image/featured/mars3.jpg'

### Mars Facts
* Visited the Mars Facts webpage [here](https://galaxyfacts-mars.com) and used Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
* Used Pandas to convert the data to a HTML table string.
* Stripped off spaces,  new line character, not-required rows and columns
* Saved the HTML string to a variable 'galaxyfacts_html'.

In [33]:
import pandas as pd

In [13]:
# Loading url to browser
url_galaxyfacts = 'https://galaxyfacts-mars.com'

# Using Pandas read_html to read the html tables from the website
# This returns a list of all tables found
tables_galaxyfacts = pd.read_html(url_galaxyfacts)

# Print to identify the required table. 
#print (tables_galaxyfacts)

# In this case , we want  table with list index 1 
# Move this HTML to a DataFrame
df = tables_galaxyfacts[1]

# converting dataframe to HTML by using pandas 'to_HTML'. Set the index to False
galaxyfacts_html = df.to_html(index=False)

# Stripping spaces,  new line character, not-required rows and columns
galaxyfacts_html = galaxyfacts_html.replace('\n','')
galaxyfacts_html = galaxyfacts_html.replace('class="dataframe"','class="table tablipede-str"')
galaxyfacts_html = galaxyfacts_html.replace('<thead>    <tr style="text-align: right;">      <th>0</th>      <th>1</th>    </tr>  </thead>','')

# Adding clean data to scrape_data dictionary  
scrape_data["galaxyfacts_html"]  = galaxyfacts_html
galaxyfacts_html


'<table border="1" class="table tablipede-str">    <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 ( Phobos &amp; Deimos )</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

### Mars Hemispheres

* Visit the astrogeology site 'https://marshemispheres.com/' to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

```python
# Example:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
]
```

In [ ]:
url_hem = 'https://marshemispheres.com/'
browser.visit(url_hem)
browser.reload()

In [13]:
hemisphere_image_urls  = []
url_hem_html = browser.html

soup_url_hem_html = BeautifulSoup(url_hem_html,'html.parser')
hem_img_divs = soup_url_hem_html.find_all('div', class_="description")
for item in hem_img_divs:
     #print (item.find('a').get('href'))
     link_text = item.find('a').text
     link_text=link_text.strip()
     
     but = browser.find_by_text(link_text)
     but.click()

     # At this point broswer has new url
     soup_full_res = BeautifulSoup(browser.html,'html.parser')
     full_res_div  = soup_full_res.find_all('div', class_="downloads")
     full_res_img_url = url_hem + full_res_div[0].find('a').get('href')
     print(full_res_img_url)
     
     url_dict = {"title":link_text, "img_url=":full_res_img_url}
     hemisphere_image_urls.append(url_dict)

     browser.visit(url_hem)
     browser.reload()



In [ ]:
hemisphere_image_urls



[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url=': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url=': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url=': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url=': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [14]:
browser.quit()

In [1]:
url = [{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [2]:

    print(url[0]['img_url'])

https://marshemispheres.com/images/full.jpg


In [3]:
print (len(url))

4
